### Package Imports

In [28]:
import pandas as pd
import numpy as np
import json
import boto3
import s3fs
from azure.storage.blob import BlobServiceClient, generate_account_sas, ResourceTypes, AccountSasPermissions
import tables
from datetime import datetime, timedelta


### Read dataset from public cloud storage

In [18]:
data_location = input("From which location we are reading data?").lower()
if data_location in ["aws","s3","amazon"]:
    AWS_S3_BUCKET = input("Your bucket address please")
    file_name = input("Your full file name please (with extension)")
    if "csv" in file_name.split("."):
        file_name = file_name.split(".")
        df = pd.read_csv(f"s3://{AWS_S3_BUCKET}/de/{file_name[0]}.csv")
    elif "parquet" or "pqt" in file_name.split("."):
        file_name = file_name.split(".")
        df = pd.read_parquet(f"s3://{AWS_S3_BUCKET}/de/{file_name[0]}.parquet")
    else:
        print("Unavailable file format")
# Not confirmed yet to be developed by reading mechanisim
elif data_location in ["azure","blob"]:
    ACCOUNT_NAME = input("Your Azure account name please")
    STORAGE_ACCOUNT_NAME = input("Your Azure account storage name please")
    ACCOUNT_KEY = input("Your Azure account key please")
    file_name = input("Your full file name please (with extension)")
    blob_name = input("Your blob name please")
    container_name = input("Your container name please")
    sas_token = generate_account_sas(
    account_name="{STORAGE_ACCOUNT_NAME}",
    account_key="{ACCOUNT_KEY}",
    resource_types=ResourceTypes(service=True),
    permission=AccountSasPermissions(read=True),
    expiry=datetime.utcnow() + timedelta(hours=1)
    )
    blob_service_client = BlobServiceClient(account_url=f"https://{ACCOUNT_NAME}.blob.core.windows.net"
                                            , credential=sas_token)
    blob_service_client.get_blob_to_path(container_name,blob_name,file_name)
    if "csv" in file_name.split("."):
        file_name = file_name.split(".")
        df = pd.read_csv(f"{file_name}")
    elif "parquet" or "pqt" in file_name.split("."):
        file_name = file_name.split(".")
        df = pd.read_parquet(f"{file_name}")
    else:
        print("Unavailable file format")
else:
    print("no connector available from given file source location")



### General Details over Dataset

In [43]:
# Column data types
df.dtypes

adjective      object
category       object
created_at     object
id            float64
modifier       object
name           object
price         float64
dtype: object

In [42]:
# Non numerical columns
df.describe(exclude=[np.number])

,adjective,category,created_at,modifier,name
count,1926,2198,2198,1432,2198
unique,13,10,2198,9,1011
top,reflective,instrument,2014-01-15 21:36:09,storage_unit,apparatus
freq,168,239,1,186,13


In [44]:
# All columns general stats
df.describe(include='all')

,adjective,category,created_at,id,modifier,name,price
count,1926,2198,2198,2198.000000,1432,2198,2198.000000
unique,13,10,2198,NaN,9,1011,NaN
top,reflective,instrument,2014-01-15 21:36:09,NaN,storage_unit,apparatus,NaN
freq,168,239,1,NaN,186,13,NaN
mean,NaN,NaN,NaN,1991.293904,NaN,NaN,98.157420
std,NaN,NaN,NaN,1163.964428,NaN,NaN,156.059937
min,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000
25%,NaN,NaN,NaN,981.250000,NaN,NaN,13.750000
50%,NaN,NaN,NaN,2008.000000,NaN,NaN,42.000000
75%,NaN,NaN,NaN,2994.500000,NaN,NaN,115.700000
